In [1]:
%load_ext dotenv
%dotenv

In [2]:
from VLA_LLM import prompts
from VLA_LLM.agents import ChatConversationalVLAAgent
from VLA_LLM.tools import AppointmentSchedulerAndAvailabilityTool
from VLA_LLM.tools import CurrentTimeTool
from VLA_LLM.community_info import EXAMPLE_COMMUNITY_DATA
from VLA_LLM.community_info import community_dict_to_prompt

In [3]:
tools = [CurrentTimeTool(), AppointmentSchedulerAndAvailabilityTool()]

In [4]:
# most deterministic results
temperature = 0

In [5]:
community_info = community_dict_to_prompt(EXAMPLE_COMMUNITY_DATA)

In [6]:
print(community_info)

Parking options:
  1. Parking option 1
     Parking type: Parking Available
     Parking fee: $100.0
     Parking fee term: Monthly
     Parking notes: noties parking available

Pet policy: Pets Allowed. Extra fee for pets: 50.0


In [7]:
# from langchain.prompts import PromptTemplate
# prompt = PromptTemplate.from_template(f"{prompts.prompt_2}\n\n{{prospect_message}}")
# prompt.format(community_info=community_info)

In [8]:
prompt_template = (
    f"{prompts.prompt_1.format(community_info=community_info)}\n\n"
    "Here is the prospect message:\n\n{prospect_message}"
)

In [9]:
print(prompt_template)

You are a leasing agent at a large multifamily apartment building talking with a prospect interested in renting one of your units. Your job is to answer questions about the community based on the information below. You should also try to gather their preferences like budget, move-in date, and desired layout, while nudging them to book a tour.

Here is information about the property that you can answer questions about:

Parking options:
  1. Parking option 1
     Parking type: Parking Available
     Parking fee: $100.0
     Parking fee term: Monthly
     Parking notes: noties parking available

Pet policy: Pets Allowed. Extra fee for pets: 50.0

If the prospect provides an appointment time or date, first call the get_current_time tool to get the current day of week and time. Then pass the appointment time or date to the appointment scheduler tool in paraphrase the result to the prospect conversationally.

If they do not provide an appointment time or date, do not call the tool.

*Note*:

In [10]:
agent = ChatConversationalVLAAgent(temperature, tools)

In [11]:
def run_agent(message):
    return agent.agent_chain.run(input=prompt_template.format(prospect_message=message))

### Test Conversation 1

Basic question about the community.

In [12]:
run_agent("What is the parking situation?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "The parking options at our apartment building include parking available. There is a monthly parking fee of $100.0. Please let me know if you have any other questions or if there's anything else I can assist you with."
}

> Finished chain.


"The parking options at our apartment building include parking available. There is a monthly parking fee of $100.0. Please let me know if you have any other questions or if there's anything else I can assist you with."

In [13]:
run_agent("i have a dog - can i bring him?")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: Yes, pets are allowed at our apartment building. There is an extra fee of $50.0 for pets. If you have any other questions or if there's anything else I can assist you with, please let me know.

### Test Conversation 2

Asking about an available appointment date.

In [14]:
run_agent("Can I schedule a tour for 7/26?")



> Entering new AgentExecutor chain...
{
    "action": "get_current_time",
    "action_input": ""
}
Observation: Tuesday 2023-07-25 22:24:30
Thought:{
    "action": "appointment_scheduler_availability",
    "action_input": "7/26"
}
Observation: Appointment date is not in the correct format. The agent should provide it in YYYY-MM-DD format, then run again.
Thought:{
    "action": "Final Answer",
    "action_input": "I apologize for the inconvenience. Could you please provide the desired appointment date in the format YYYY-MM-DD? Once I have that information, I'll be able to check the availability and schedule a tour for you."
}

> Finished chain.


"I apologize for the inconvenience. Could you please provide the desired appointment date in the format YYYY-MM-DD? Once I have that information, I'll be able to check the availability and schedule a tour for you."

### Test Conversation 3

Asking about an unavailable appointment date.

In [15]:
run_agent("You do have any available appointments on 7/28?")



> Entering new AgentExecutor chain...
{
    "action": "appointment_scheduler_availability",
    "action_input": "7/28"
}
Observation: Appointment date is not in the correct format. The agent should provide it in YYYY-MM-DD format, then run again.
Thought:{
    "action": "Final Answer",
    "action_input": "I apologize for the inconvenience. Could you please provide the desired appointment date in the format YYYY-MM-DD? Once I have that information, I'll be able to check the availability and schedule a tour for you."
}

> Finished chain.


"I apologize for the inconvenience. Could you please provide the desired appointment date in the format YYYY-MM-DD? Once I have that information, I'll be able to check the availability and schedule a tour for you."